In [1]:
from datetime import datetime, timezone
import orjson
import aiohttp
import asyncio
import pandas as pd
import time

async def get(url):
	async with aiohttp.ClientSession(json_serialize=orjson.dumps) as session:
		async with session.get(url) as resp:
			data = await resp.json(loads=orjson.loads)
			return data

# timeframe being in seconds (5, 60, 300, 900, 3600, 14400, 86400)    
async def determine_count(year, month, day, hour, minute, timeframe=60):
    starting_timestamp = int(datetime(year=2021, month=10, day=1, hour=0, minute=0).timestamp())
    current_time = int(time.time())
    return (current_time - starting_timestamp) // timeframe
    
async def aggregate_klines(underlying, granularity, count):

    aggregate = pd.DataFrame(columns=['startTime', 'time','open', 'high', 'low', 'close', 'volume'])
    step = 0
    
    # get current timestep to the nearest round minute to align with ftx candle data aggregation (i think)
    # to give us a starting timestep 

    current_utc_timestamp = time.time()

    most_recent_candle = current_utc_timestamp - (current_utc_timestamp % granularity)

    # instead of calculating the remaining timesteps pre-1st request, just send the request, count how many 
    # instances of OHLC data we get back, deduce that from count (its usually 1500), and run it back turbo.

    while aggregate.shape[0] < count:
        if step == 0:            
            data = await get(url = f"https://ftx.com/api/markets/{underlying}/candles?resolution={granularity}&end_time={most_recent_candle}")
        else:
            end = int(most_recent_candle - (granularity * step))
            data = await get(url=f"https://ftx.com/api/markets/{underlying}/candles?resolution={granularity}&end_time={end}") 

        data = data['result'][::-1]
        
        current = pd.DataFrame(data=data)
        aggregate = aggregate.append(current, ignore_index=True)
        step += len(data)
              
    # check if current step is greater than count, and trim off excess data.
        if step > count:
            aggregate = aggregate.iloc[:count+1]
                  
    return aggregate

count = await determine_count(timeframe=60, year=2021, month=10, day=1, hour=0, minute=0)
await aggregate_klines(underlying='BTC-PERP', granularity=60, count=count)



,startTime,time,open,high,low,close,volume
0,2021-11-18T16:17:00+00:00,1.637252e+12,58274.0,58293.0,58203.0,58221.0,2.610217e+06
1,2021-11-18T16:16:00+00:00,1.637252e+12,58357.0,58387.0,58271.0,58274.0,5.798330e+06
2,2021-11-18T16:15:00+00:00,1.637252e+12,58429.0,58509.0,58265.0,58363.0,9.701192e+06
3,2021-11-18T16:14:00+00:00,1.637252e+12,58377.0,58512.0,58377.0,58429.0,6.769256e+06
4,2021-11-18T16:13:00+00:00,1.637252e+12,58348.0,58403.0,58348.0,58377.0,3.820909e+06
...,...,...,...,...,...,...,...
69853,2021-10-01T04:04:00+00:00,1.633061e+12,43608.0,43654.0,43589.0,43621.0,1.461702e+06
69854,2021-10-01T04:03:00+00:00,1.633061e+12,43603.0,43625.0,43547.0,43608.0,6.408331e+06
69855,2021-10-01T04:02:00+00:00,1.633061e+12,43664.0,43674.0,43599.0,43603.0,2.104156e+06
69856,2021-10-01T04:01:00+00:00,1.633061e+12,43677.0,43685.0,43658.0,43661.0,6.231602e+05
